<a href="https://colab.research.google.com/github/davidogm/DataScience/blob/main/ITBD/trabajos2024/Crime_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Elección del Dataset

El dataset elegido para esta práctica ha sido el de *[Crime Data Analysis](!https://www.kaggle.com/datasets/candacegostinski/crime-data-analysis?select=Crime_Data_from_2020_to_Present.csv)* de Kaggle, ya que este cuenta con casi un millón de registros (desde 2020 hasta Abril de 2024) y 28 columnas con una gran diversidad de campos distintos.

---

# 📦 Paquetes Necesarios

Para realizar la actividad se usarán diversos paquetes que facilitarán el manejo de los datos y el desarrollo del proyecto.

## 1. 📁 **virtualenv**

El paquete `virtualenv` permite crear un entorno dentro de nuestro proyecto de forma aislada, permitiendo usar otras versiones de Python y paquetes sin afectar el entorno global.

### Instalación:

```bash
pip install virtualenv
```

### Activación:
- Windows:
```bash
virtualenv env
env\Scripts\activate
```

- Linux:
```bash
virtualenv env
source env/bin/activate
```

## 2. 🐼 Pandas

## 3. 🔍 Kaggle API
Esta API permite descargar datasets de Kaggle de forma sencilla, ya sea mediante CLI o desde código en Python. Para evitar configuraciones adicionales con tokens y accesos a la API, la descarga se realizará mediante CLI.

```bash
kaggle datasets download -d candacegostinski/crime-data-analysis --unzip
```

## 4. 🔥 PySpark

## 5. 📊 Matplotlib

## 6. 📈 Seaborn

## 7. 📍 Plotly
`Plotly` permite crear gráficos interactivos. Para su correcto funcionamiento en notebooks, es necesario instalar nbformat.

## 8. 🔧 re

## 9. 🕒 time

## 10. ➗ numpy

---

# Carga del dataset

Para realizar una análisis más exhaustivo durante la practica, el dataset será cargado de diversas formas con el fin de analizar también los tiempos de carga:

1. Pandas

    1.1 De forma local

    1.2. Kaggle API

    1.3. Google Drive
    
3. PySpark
4. Dask


### Imports

In [ ]:
import time
import pandas as pd
import dask.dataframe as dd
import re
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from matplotlib import cm
import folium
from folium.plugins import HeatMap
import geopandas as gpd

from google.colab import drive
drive.mount('/content/drive')

df_path = '/content/drive/MyDrive/ITBD/Crime_Data_from_2020_to_Present.csv'
geojson_path = '/content/drive/MyDrive/ITBD/los-angeles.geojson'

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Mounted at /content/drive


## **1. Pandas**

In [ ]:
%%time
## De forma local

start_time_to_load_from_pandas = time.time()
df_pd_local = pd.read_csv(df_path)
end_time_to_load_from_pandas = time.time()

CPU times: user 5.65 s, sys: 1.1 s, total: 6.75 s
Wall time: 9.34 s


In [ ]:
%%time
## Usando la API de Kaggle

start_time_to_load_from_kaggle = time.time()
# kaggle datasets download -d candacegostinski/crime-data-analysis --unzip
df_pd_kaggle = pd.read_csv(df_path)
end_time_to_load_from_kaggle = time.time()

CPU times: user 5.16 s, sys: 879 ms, total: 6.04 s
Wall time: 6.49 s


## **2. PySpark**

In [ ]:
%%time

# start_time_to_load_from_pyspark = time.time()
# data = spark.read.csv('Crime_Data_from_2020_to_Present.csv', header = True)
# end_time_to_load_from_pyspark = time.time()

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 7.87 µs


## **3. Dask**

In [ ]:
%%time

start_time_to_load_from_dask = time.time()
df_dask = dd.read_csv(df_path, sample=25000000)
end_time_to_load_from_dask = time.time()

CPU times: user 63.7 ms, sys: 155 ms, total: 219 ms
Wall time: 352 ms


In [ ]:
total_time_from_pandas = end_time_to_load_from_pandas - start_time_to_load_from_pandas
total_time_from_kaggle = end_time_to_load_from_kaggle - start_time_to_load_from_kaggle
# total_time_from_spark = end_time_to_load_from_spark - start_time_time_load_from_dask
total_time_from_dask = end_time_to_load_from_dask - start_time_to_load_from_dask

In [ ]:
time_list = [
    {'Library': 'Pandas', 'Time': total_time_from_pandas},
    {'Library': 'Kaggle','Time': total_time_from_kaggle},
    {'Library': 'Dask', 'Time': total_time_from_dask}
]

df = pd.DataFrame(columns=['Library', 'Time'], data=time_list)
print(df)

  Library      Time
0  Pandas  9.340553
1  Kaggle  6.487647
2    Dask  0.351701


---

# Descripción de los datos

Para simplificar nombres de variables vamos a crear una nueva llamda *df*.

In [ ]:
df = pd.read_csv(df_path)

In [ ]:
df.head(2)

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628


## Columnas del dataframe

In [ ]:
df_cols = df.columns
print(df_cols)

Index(['DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME',
       'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1',
       'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'Cross Street', 'LAT',
       'LON'],
      dtype='object')


## Tamaño del dataframe

In [ ]:
df_size = len(df)
print(df_size)

955339


## Descripción general de dataframe

In [ ]:
df_memory_size = os.path.getsize(df_path)
print(f'El fichero csv pesa: {df_memory_size} B ~ {round(df_memory_size / (1024 * 1024), 2)} MB')

El fichero csv pesa: 243690902 B ~ 232.4 MB


In [ ]:
shape = df.shape
print(f'Rows: {shape[0]} | Columns: {shape[1]}')

Rows: 955339 | Columns: 28


In [ ]:
df.describe()

,DR_NO,TIME OCC,AREA,Rpt Dist No,Part 1-2,Crm Cd,Vict Age,Premis Cd,Weapon Used Cd,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LAT,LON
count,9.553390e+05,955339.000000,955339.000000,955339.000000,955339.000000,955339.000000,955339.000000,955327.000000,325019.000000,955328.000000,68466.000000,2294.000000,64.00000,955339.000000,955339.000000
mean,2.191355e+08,1338.453032,10.718033,1118.243545,1.407805,500.667710,29.383110,306.255200,363.741547,500.415756,958.107265,984.102005,991.21875,33.993372,-118.074280
std,1.261589e+07,652.186527,6.100431,610.098189,0.491427,206.867837,21.909695,217.844195,123.606085,206.661318,110.316767,51.662269,27.06985,1.660523,5.755746
min,8.170000e+02,1.000000,1.000000,101.000000,1.000000,110.000000,-4.000000,101.000000,101.000000,110.000000,210.000000,310.000000,821.00000,0.000000,-118.667600
25%,2.105125e+08,900.000000,6.000000,621.000000,1.000000,331.000000,0.000000,101.000000,311.000000,331.000000,998.000000,998.000000,998.00000,34.014500,-118.430700
50%,2.207106e+08,1420.000000,11.000000,1142.000000,1.000000,442.000000,30.000000,203.000000,400.000000,442.000000,998.000000,998.000000,998.00000,34.059000,-118.322500
75%,2.307171e+08,1900.000000,16.000000,1618.000000,2.000000,626.000000,45.000000,501.000000,400.000000,626.000000,998.000000,998.000000,998.00000,34.164900,-118.274000
max,2.499187e+08,2359.000000,21.000000,2199.000000,2.000000,956.000000,120.000000,976.000000,516.000000,956.000000,999.000000,999.000000,999.00000,34.334300,0.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 955339 entries, 0 to 955338
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           955339 non-null  int64  
 1   Date Rptd       955339 non-null  object 
 2   DATE OCC        955339 non-null  object 
 3   TIME OCC        955339 non-null  int64  
 4   AREA            955339 non-null  int64  
 5   AREA NAME       955339 non-null  object 
 6   Rpt Dist No     955339 non-null  int64  
 7   Part 1-2        955339 non-null  int64  
 8   Crm Cd          955339 non-null  int64  
 9   Crm Cd Desc     955339 non-null  object 
 10  Mocodes         818664 non-null  object 
 11  Vict Age        955339 non-null  int64  
 12  Vict Sex        825294 non-null  object 
 13  Vict Descent    825284 non-null  object 
 14  Premis Cd       955327 non-null  float64
 15  Premis Desc     954770 non-null  object 
 16  Weapon Used Cd  325019 non-null  float64
 17  Weapon Des

---

# Preprocesamiento

## Búsqueda de nulos

La función *df.insnull().any()* mostrará todas las columnas, y en caso de que alguna de éstas contenga un nulo, pondrá un True

In [ ]:
null_values = df.isnull().any()
print(null_values)

DR_NO             False
Date Rptd         False
DATE OCC          False
TIME OCC          False
AREA              False
AREA NAME         False
Rpt Dist No       False
Part 1-2          False
Crm Cd            False
Crm Cd Desc       False
Mocodes            True
Vict Age          False
Vict Sex           True
Vict Descent       True
Premis Cd          True
Premis Desc        True
Weapon Used Cd     True
Weapon Desc        True
Status             True
Status Desc       False
Crm Cd 1           True
Crm Cd 2           True
Crm Cd 3           True
Crm Cd 4           True
LOCATION          False
Cross Street       True
LAT               False
LON               False
dtype: bool


### Los nulos pueden ser tratados de 2 formas:

#### **1. Borrándolos**

Para borrar los valores nulos de un df, se puede usar la función *df.dropna()*

In [ ]:
df.dropna().isnull().any()

,0
DR_NO,False
Date Rptd,False
DATE OCC,False
TIME OCC,False
AREA,False
AREA NAME,False
Rpt Dist No,False
Part 1-2,False
Crm Cd,False
Crm Cd Desc,False


#### **2. Rellenándolos**

Para rellenar los valores nulos se puede usar la función *df.fillna()*. Para esta función, el valor por el cual se sustituye puede ser variado. Puede ser un valor fijo como un 0 o un string, también puede ser el último valor no nulo observado, es decir, el valor inmediatamente anterior o el siguiente valor no nulo, es decir, el valor inmediatamente siguiente.

Es necesario recalcar que hay *axis* deber ser igual 1, para indicar que trabaje a nivel de columna.

In [ ]:
####################### La comento porque tarda mucho
df.fillna(method='ffill', axis=1).isna().any()

<ipython-input-18-c4479a4e3e4d>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1).isna().any()


,0
DR_NO,False
Date Rptd,False
DATE OCC,False
TIME OCC,False
AREA,False
AREA NAME,False
Rpt Dist No,False
Part 1-2,False
Crm Cd,False
Crm Cd Desc,False


## Búsqueda de duplicados

De forma muy similar a la función **isna** o **isnull**, Pandas tiene la función **duplicated** que muestra si alguna linea está duplicada. Para eliminar los duplicados se puede usar la función **drop_duplicated**.

In [ ]:
total_duplicated = df.duplicated().sum()
print(f'El df contiene {total_duplicated} registros repetidos')

El df contiene 0 registros repetidos


---

## Explicación de datos

#### 📄 Funciones de Filtrado para DataFrames

Este documento describe dos funciones de filtrado y una variable de rangos de fechas que se pueden utilizar para analizar y segmentar datos de un DataFrame en Python.

##### 📅 1. **`dates`**

`dates` es una lista de tuplas que contiene intervalos de fechas y el año correspondiente.

##### 🗃️ 2. **`filter_df_by_date(df, start_date, end_date)`**
Esta función filtra un DataFrame (df) basado en un rango de fechas (start_date y end_date). Devuelve un subconjunto del DataFrame original con filas que cumplen la condición de estar dentro del rango especificado.

##### 🔍 3. **`filter_df_by_crime(df, crime)`**
Esta función filtra un DataFrame (df) basado en el tipo de crimen especificado (crime). Devuelve un subconjunto del DataFrame original con las filas que contengan la descripción del crimen en la columna `Crm Cd Desc`.

#### 4. Borrado de espacios en las calles para facilitar la visualización

In [ ]:
dates = [
    ('03/01/2020', '03/01/2021', '2020'),
    ('03/01/2021', '03/01/2022', '2021'),
    ('03/01/2022', '03/01/2023', '2022'),
    ('03/01/2023', '03/01/2024', '2023'),
    ('03/01/2024', '03/01/2025', '2024')
]

def filter_df_by_date(df, start_date, end_date):
    return df[(df['Single date'] < end_date) & (df['Single date'] >= start_date)]

def filter_df_by_crime(df, crime):
    return df[df['Crm Cd Desc'].str.contains(crime)]

df['LOCATION'] = df['LOCATION'].apply(lambda x: re.sub(r'\s+', ' ', x))

Para procesar las fechas de una manera mejor se crea una nueva columna en el dataframe que solo contiene la fecha, sin hora.

In [ ]:
df['Single date'] = pd.to_datetime(df['Date Rptd'], format='%m/%d/%Y %I:%M:%S %p')
df['Single date'] = df['Single date'].dt.date

### Total de crímenes por área desde 03/01/2020 hasta 04/24/2024

In [ ]:
total_areas = df['AREA NAME'].nunique()
print(f'Hay un total de {total_areas} áreas dentro de LA.')
print('-'*50, '\n')

Hay un total de 21 áreas dentro de LA.
-------------------------------------------------- 



### Total de crímenes y los 20 tipos de crímenes más cometidos

In [ ]:
total_crimes = df['Crm Cd Desc'].nunique()
print(f'Hay un total de {total_crimes} crímenes distintos.')
print('-'*50, '\n')

crimes_count = df.groupby(['AREA NAME', 'Crm Cd Desc']).size().reset_index(name='Incidencias')

fig = px.bar(
    crimes_count,
    x='Crm Cd Desc',
    y='Incidencias',
    color='AREA NAME',
    title='Total de crímenes agrupado por área 2020-2024',
    labels={'Crm Cd Desc': 'Tipo de crimen', 'Incidencias': 'Incidencias'},
    hover_data={'Incidencias': True}
)

fig.update_layout(
    xaxis_title='Tipo de crimen',
    yaxis_title='Incidencias',
    width=1200,
    height=1200,
)

fig.show()

Hay un total de 139 crímenes distintos.
-------------------------------------------------- 



### Los 20 crímenes más cometidos por año

In [ ]:
all_crimes = df['Crm Cd Desc'].value_counts().index.tolist()[0:20]
colormap = cm.get_cmap('tab20', 20)

color_mapping = {crime: f'rgba{tuple(map(lambda x: int(x*255), colormap(i)[:3])) + (1,)}' for i, crime in enumerate(all_crimes)}

fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=[f'Top 20 crímenes más cometidos en {year}' for _, _, year in dates],
    vertical_spacing=0.25, horizontal_spacing=0.1
)

years_result = []

for i, (start, end, year) in enumerate(dates):
    start_date = datetime.strptime(start, '%m/%d/%Y').date()
    end_date = datetime.strptime(end, '%m/%d/%Y').date()

    filtered_data = filter_df_by_date(df, start_date, end_date)['Crm Cd Desc'].value_counts()[0:20]

    row, col = divmod(i, 2)
    fig.add_trace(
        go.Bar(
            x=filtered_data.index,
            y=filtered_data.values,
            marker_color=[color_mapping.get(crime, 'rgba(128,128,128,1)') for crime in filtered_data.index],
            hovertemplate='<b>%{x}</b><br>Incidencias: %{y}<extra></extra>',
        ),
        row=row + 1, col=col + 1
    )

    years_result.append(filtered_data)

fig.update_layout(
    title_text='Comparativa de crímenes más comunes por año',
    showlegend=False,
    width=1200,
    height=1600,
    margin=dict(l=50, r=50, b=1, t=100),
    font=dict(size=10),
)

fig.update_xaxes(tickangle=-45)

fig.show()

<ipython-input-24-c807f92fbacd>:2: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



### Media de cada crimen a lo largo de los 4 años (de los 20 más comunes)

In [ ]:
combined_crimes = pd.DataFrame(years_result)

mean_crimes = combined_crimes.mean(axis=0)

mean_crimes_df = mean_crimes.reset_index()
mean_crimes_df.columns = ['Crm Cd Desc', 'Mean Occurrences']

fig = px.bar(
    mean_crimes_df,
    x='Crm Cd Desc',
    y='Mean Occurrences',
    title='Media de cada crimen a lo largo de los 4 años (de los 20 más comunes)',
    labels={'Crm Cd Desc': 'Tipo de Crimen', 'Mean Occurrences': 'Media de Ocurrencias'},
    color='Crm Cd Desc',
    hover_data={'Mean Occurrences': ':.0f'},
)

fig.update_layout(width=1000, height=1000)

fig.show()

### Crímen más cometido por año

In [ ]:
for year_result, current_year in zip(years_result, ['2020', '2021', '2022', '2023', '2024']):
    temp_df = pd.DataFrame(year_result.reset_index()).sort_values(by='count', ascending=False)

    print(f"El crimen más cometido en {current_year} fue {temp_df.loc[0, 'Crm Cd Desc']} con {temp_df.loc[0, 'count']} incidencias.")

El crimen más cometido en 2020 fue VEHICLE - STOLEN con 21663 incidencias.
El crimen más cometido en 2021 fue VEHICLE - STOLEN con 23894 incidencias.
El crimen más cometido en 2022 fue VEHICLE - STOLEN con 24517 incidencias.
El crimen más cometido en 2023 fue VEHICLE - STOLEN con 24828 incidencias.
El crimen más cometido en 2024 fue VEHICLE - STOLEN con 7214 incidencias.


### Área con más crímenes (en los 4 años)

In [ ]:
all_years_crimes_count = df['AREA NAME'].value_counts()
area = all_years_crimes_count.idxmax()
times = all_years_crimes_count.max()

print(f'El área más afectada en los 4 años ha sido {area} con {times} incidentes.')

El área más afectada en los 4 años ha sido Central con 64737 incidentes.


### Áreas con su correspondiente crimen más común

In [ ]:
most_common_crime_per_area = df.groupby('AREA NAME')['Crm Cd Desc'].agg(
    Crime=lambda x: x.value_counts().idxmax(),
    Amount=lambda x: x.value_counts().max()
).reset_index()


fig = px.bar(
	most_common_crime_per_area,
	x='AREA NAME',
	y='Amount',
	title='Crímen más común por área (en los 4 años)',
	labels={'AREA NAME': 'Área', 'Crime': 'Crímen', 'Amount': 'Total'},
	color='AREA NAME',
	hover_data={'Crime': True, 'Amount': ':.0f'}
)

fig.update_layout(width=1000, height=700)
fig.update_xaxes(tickangle=-45)

fig.show()

### Edad más afectada

In [ ]:
# Filtramos age != 0 para evitar objetos

most_common_age = df[df['Vict Age'] != 0]['Vict Age'].value_counts().idxmax()
amount = df['Vict Age'].value_counts().max()

print(f'La edad más afectada es {most_common_age} años, con un total de {amount} incidentes.')
print('-'*50, '\n')


La edad más afectada es 30 años, con un total de 245607 incidentes.
-------------------------------------------------- 



### Edad más afectada por área

In [ ]:
most_common_age_per_area = df[df['Vict Age'] != 0].groupby('AREA NAME')['Vict Age'].agg(
    Age=lambda x: x.value_counts().idxmax(),
    Amount=lambda x: x.value_counts().max()
).reset_index()

fig = px.bar(
	most_common_age_per_area,
	title='Edad más afectada por área',
	x='AREA NAME',
	y='Amount',
	labels={'AREA NAME': 'Área', 'Amount': 'Total', 'Age': 'Año'},
	color='AREA NAME',
	hover_data={'Age': True, 'Amount': ':.0f'},
)

fig.update_layout(width=700, height=700)
fig.show()

### Sexo más afectado

In [ ]:
most_common_sex = df[(df['Vict Sex'].notna()) & (df['Vict Sex'].isin(['M', 'F']))]['Vict Sex'].value_counts().idxmax()
print(f'El sexo más afectado por crímenes es: {"Hombre" if most_common_sex == "M" else "Mujer"}')
print('-'*50)

El sexo más afectado por crímenes es: Hombre
--------------------------------------------------


### Arma más utilizada (total de crímenes)

In [ ]:
most_common_weapon = df['Weapon Desc'].value_counts().idxmax()
amount = df['Weapon Desc'].value_counts().max()

print(f'El arma más utilizada es {most_common_weapon}, reportado {amount} veces.')
print('-'*50)

El arma más utilizada es STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE), reportado 174057 veces.
--------------------------------------------------


### Arma más utilizada por área

In [ ]:
most_common_weapon_per_area = df.groupby('AREA NAME')['Weapon Desc'].agg(
    Weapon=lambda x: x.value_counts().idxmax(),
    Amount=lambda x: x.value_counts().max()
).reset_index()

fig = px.bar(
	most_common_weapon_per_area,
	x='AREA NAME',
	y='Amount',
	title='Arma más utilizada por área',
	labels={'AREA NAME': 'Área', 'Amount': 'Total'},
	color='AREA NAME',
	hover_data={'Weapon': True, 'Amount': ':.0f'}
)

fig.update_layout(width=700, height=700)
fig.show()

### Crímenes por calle y año

In [ ]:
for i in range(len(dates)):
    start_date = datetime.strptime(dates[i][0], '%m/%d/%Y').date()
    end_date = datetime.strptime(dates[i][1], '%m/%d/%Y').date()

    temp_df = filter_df_by_date(df, start_date=start_date, end_date=end_date)
    temp_df_counts = temp_df['LOCATION'].value_counts()

    street = temp_df_counts.idxmax()
    times = temp_df_counts.max()

    print(f'En {dates[i][2]}, la calle con más crímenes fue "{street}" con {times} incidentes.')

En 2020, la calle con más crímenes fue "6TH ST" con 332 incidentes.
En 2021, la calle con más crímenes fue "800 N ALAMEDA ST" con 487 incidentes.
En 2022, la calle con más crímenes fue "10200 SANTA MONICA BL" con 530 incidentes.
En 2023, la calle con más crímenes fue "11800 SANTA MONICA BL" con 733 incidentes.
En 2024, la calle con más crímenes fue "700 S FIGUEROA ST" con 221 incidentes.


### Calle más afectada (en los 4 años)

In [ ]:
most_affected_street = df['LOCATION'].value_counts().idxmax()
amount = df['LOCATION'].value_counts().max()

print(f'La calle más afectada es "{most_affected_street}" con {amount} incidentes.')

La calle más afectada es "800 N ALAMEDA ST" con 1916 incidentes.


### Crímen más cometido por calle y año (en las 20 calles más afectadas)

In [ ]:
most_affected_streets = df['LOCATION'].value_counts().head(20).index

fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=[f'Crímen más cometidos por calle en {year}' for _, _, year in dates],
    vertical_spacing=0.25, horizontal_spacing=0.3
)

for i in range(len(dates)):
    start_date = datetime.strptime(dates[i][0], '%m/%d/%Y').date()
    end_date = datetime.strptime(dates[i][1], '%m/%d/%Y').date()

    temp_df = filter_df_by_date(df, start_date=start_date, end_date=end_date)
    filtered_temp_df = temp_df[temp_df['LOCATION'].isin(most_affected_streets)]

    crime_per_location = filtered_temp_df.groupby('LOCATION')['Crm Cd Desc'].agg(
        Crime=lambda x: x.value_counts().idxmax(),
        Amount=lambda x: x.value_counts().max()
    ).reset_index()

    heatmap_street_crime = pd.crosstab(
        index=filtered_temp_df['LOCATION'],
        columns=filtered_temp_df[filtered_temp_df['Crm Cd Desc'].str.contains('THEFT')]['Crm Cd Desc']
    )

    row, col = divmod(i, 2)


    fig.add_trace(
        go.Heatmap(
            z=heatmap_street_crime.values,
            x=heatmap_street_crime.columns,
            y=heatmap_street_crime.index,
            colorscale='YlGnBu',
            zmin=0,
            zmax=15,
            hoverongaps=False
        ),
        row=row + 1, col=col + 1
    )

    fig.update_layout(
        title_text='Comparativa de crímenes por calle y por año',
        showlegend=False,
        width=1200,
        height=1600,
        margin=dict(l=50, r=50, b=1, t=100),
        font=dict(size=10),
    )

    fig.update_xaxes(tickangle=-45)

fig.show()

### Representación en mapa de los robos por calle en 2021

In [ ]:
## Cargamos el mapa de LA

la_map  = gpd.read_file(geojson_path)
la_map = la_map[['geometry']]
m = folium.Map(location=[34.0522, -118.2437], zoom_start=11)

In [ ]:
from IPython.display import display

start_date = datetime.strptime(dates[1][0], '%m/%d/%Y').date()
end_date = datetime.strptime(dates[1][1], '%m/%d/%Y').date()

# Filtramos el año 2021
year_filtered_df = filter_df_by_date(df, start_date, end_date)

# Filtramos por robos (THEFT)
crime_filtered_df = filter_df_by_crime(year_filtered_df, 'THEFT')

# Filtramos por calle
street_filtered_df = crime_filtered_df[crime_filtered_df['LOCATION'].isin(most_affected_streets)].loc[:, ['LAT', 'LON']].dropna().values.tolist()

HeatMap(street_filtered_df).add_to(m)
display(m)

### Representación en mapa de los robos relacionados con vehiculos por calle en 2021

In [ ]:
crime_filtered_df = filter_df_by_crime(year_filtered_df, 'VEHICLE')

# Filtramos por calle
street_filtered_df = crime_filtered_df[crime_filtered_df['LOCATION'].isin(most_affected_streets)].loc[:, ['LAT', 'LON']].dropna().values.tolist()

HeatMap(street_filtered_df).add_to(m)
display(m)

### Representación de robos cometidos por año en 2021

In [ ]:
crime_filtered_df = filter_df_by_crime(year_filtered_df, 'THEFT')

# Filtramos por area
area_grouped_df = crime_filtered_df.groupby('AREA NAME').value_counts().reset_index().loc[: 'LAT', 'LON'].dropna().values.tolist()

HeatMap(area_grouped_df).add_to(m)
display(m)

---

## Predicciones usando árboles

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics

#### Robos en `Central`

Agrupamos los crímenes por Central y robos

In [ ]:
filtered_df = df[(df['Crm Cd Desc'].str.contains('THEFT')) & (df['AREA NAME'] == 'Central')]
grouped_df = filtered_df.groupby('Single date').size().reset_index(name='número_crimenes').sort_values(by='Single date')

Para hacer más preciso el model, separamos por día, mes, año y día de la semana

In [ ]:
grouped_df['año'] = pd.to_datetime(grouped_df['Single date']).dt.year
grouped_df['mes'] = pd.to_datetime(grouped_df['Single date']).dt.month
grouped_df['día'] = pd.to_datetime(grouped_df['Single date']).dt.day
grouped_df['día_semana'] = pd.to_datetime(grouped_df['Single date']).dt.dayofweek

In [ ]:
X = grouped_df[['año', 'mes', 'día', 'día_semana']]
y = grouped_df[['número_crimenes']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [ ]:
y_pred = model.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'Error cuadrático medio (RMSE): {rmse}')

Error cuadrático medio (RMSE): 5.658074202498726


In [ ]:
y_pred = y_pred.values if isinstance(y_pred, pd.DataFrame) else y_pred
y_test = y_test.values if isinstance(y_test, pd.DataFrame) else y_test

y_pred = y_pred.ravel()
y_test = y_test.ravel()

comparison = pd.DataFrame({'Año': X_test['año'], 'Mes': X_test['mes'], 'Día': X_test['día'], 'Predicción': y_pred, 'Real': y_test, 'Error': y_test - y_pred})

In [ ]:
fig = go.Figure()

comparison_fig = comparison[:10]

fig.add_trace(go.Scatter(
    x=comparison_fig['Año'],
    y=comparison_fig['Predicción'],
    mode='lines+markers',
    name='Predicción',
    marker=dict(color='blue'),
    line=dict(width=2)
))

fig.add_trace(go.Scatter(
    x=comparison_fig['Año'],
    y=comparison_fig['Real'],
    mode='lines+markers',
    name='Real',
    marker=dict(color='red'),
    line=dict(width=2)
))

fig.update_layout(
    title='Predicciones vs Valores Reales de Crímenes',
    xaxis_title='Índice',
    yaxis_title='Número de Crímenes',
    template='plotly_white'
)

fig.show()

### Robos en `Central` en el futuro

In [ ]:
start_date = pd.to_datetime('2024-05-01')
end_date = pd.to_datetime('2024-12-31')

future_dates = pd.date_range(start=start_date, end=end_date, freq='D')

future_df = pd.DataFrame({'Single date': future_dates})

In [ ]:
future_df['año'] = future_df['Single date'].dt.year
future_df['mes'] = future_df['Single date'].dt.month
future_df['día'] = future_df['Single date'].dt.day
future_df['día_semana'] = pd.to_datetime(future_df['Single date']).dt.dayofweek

In [ ]:
future_predictions = model.predict(future_df[['año', 'mes', 'día', 'día_semana']])

future_df['Predicción_crimenes'] = future_predictions

comparison = pd.DataFrame({'Año': future_df['año'], 'Mes': future_df['mes'], 'Día': future_df['día'], 'Predicción': future_predictions})
print(comparison)

      Año  Mes  Día  Predicción
0    2024    5    1         7.0
1    2024    5    2         6.0
2    2024    5    3         6.0
3    2024    5    4         3.0
4    2024    5    5         6.0
..    ...  ...  ...         ...
240  2024   12   27         4.0
241  2024   12   28         5.0
242  2024   12   29         5.0
243  2024   12   30        11.0
244  2024   12   31        11.0

[245 rows x 4 columns]


In [ ]:
fig = px.line(
    future_df,
    x='Single date',
    y='Predicción_crimenes',
    title='Predicción de crímenes en el área "Central" en 2024',
    labels={'Single date': 'Fecha', 'Predicción_crimenes': 'Predicción de crímenes'},
    markers=True
)

fig.show()

---

## Predicciones con LLMs

In [ ]:
!pip install git+https://github.com/amazon-science/chronos-forecasting.git

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-req-build-vt0s0m2v
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-req-build-vt0s0m2v
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit ac6ee36acee1e47446cd66f72f540c87f1f1fbe4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for chronos: filename=chronos-1.2.1-py3-none-any.whl size=13927 sha256=de790505fed0a5c41b935260230355d7bce27135f4fc8f1cd988181ea624ed18
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1nlhpmm/wheels/bf/c1/65/08857e57345ef1b34ba0edb4791f1b2594943e82f34e93a4ab
Successfully built chronos


In [ ]:
import torch
from chronos import ChronosPipeline

In [ ]:
pipeline = ChronosPipeline.from_pretrained(
  "amazon/chronos-t5-tiny",
  device_map="cpu",
  torch_dtype=torch.bfloat16,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [ ]:
llm_pred = grouped_df[['Single date', 'número_crimenes']]

In [ ]:
context = torch.tensor(llm_pred["número_crimenes"])
prediction_length = 64
forecast = pipeline.predict(context, prediction_length)
forecast_index = range(len(llm_pred), len(llm_pred) + prediction_length)
low, median, high = np.quantile(forecast[0].numpy(), [0.1, 0.5, 0.9], axis=0)

In [ ]:
dates = pd.date_range(start='2020-01-01', periods=len(llm_pred), freq='D')

fig = go.Figure()

fig.add_trace(go.Scatter(x=dates, y=llm_pred["número_crimenes"], mode='lines', name='historical data', line=dict(color='royalblue')))

forecast_index = pd.date_range(start=dates[-1] + pd.Timedelta(days=1), periods=prediction_length, freq='D')  # Las fechas futuras
fig.add_trace(go.Scatter(x=forecast_index, y=median, mode='lines', name='median forecast', line=dict(color='tomato')))

fig.add_traces(go.Scatter(
    x=np.concatenate([forecast_index, forecast_index[::-1]]),
    y=np.concatenate([low, high[::-1]]),
    fill='toself',
    fillcolor='rgba(255, 99, 71, 0.3)',
    line=dict(color='rgba(255, 99, 71, 0)'),
    name='80% prediction interval'
))

fig.update_layout(title='Predicción de crímenes en el área "Central"',
                  xaxis_title='Fecha',
                  yaxis_title='Número de crímenes',
                  legend=dict(x=0, y=1))

fig.show()